In [ ]:
# # run to download mnist dataset
# from mnist import download_mnist
# download_mnist()

/nfs/ghome/live/gren/mambaforge/envs/hsicfuse-env/lib/python3.9/site-packages/sklearn/datasets/_openml.py:1002: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [2]:
from sampler_mnist import ind_X_Y_corrupt
import numpy as np
import jax
import jax.numpy as jnp
from jax import random
from tqdm.auto import tqdm
from pathlib import Path
Path("results").mkdir(exist_ok=True)
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
from all_tests import hsicfuse_test, hsic_test
from all_tests import hsicagginc1_test, hsicagginc100_test, hsicagginc200_test, hsicaggincquad_test

### MNIST vary N

In [10]:
corrupt_proportion = 0.75
rep = 100
N_values = (200, 400, 600, 800, 1000)

tests = (hsicfuse_test, hsic_test, hsicagginc1_test, hsicagginc100_test, hsicagginc200_test, hsicaggincquad_test)
outputs_mnist_n = jnp.zeros((len(tests), len(N_values), rep))
outputs_mnist_n = outputs_mnist_n.tolist()
key = random.PRNGKey(42)
seed = 42
for r in tqdm(range(len(N_values))):
    N = N_values[r]
    for i in tqdm(range(rep)):
        key, subkey = random.split(key)
        X, Y = ind_X_Y_corrupt(key, N, corrupt_proportion)
        key, subkey = random.split(key)
        seed += 1
        for j in range(len(tests)):
            test = tests[j]
            outputs_mnist_n[j][r][i] = test(X, Y, subkey, seed)

outputs_mnist_n = jnp.array(outputs_mnist_n)   
power = jnp.array([[jnp.mean(outputs_mnist_n[j][r])for r in range(len(N_values))] for j in range(len(tests))])
np.save("results/mnist_n.npy", power)
np.save("results/mnist_n_x_axis.npy", N_values)

print("corrupt proportion :", corrupt_proportion)
print("sample size :", N_values)
for t in range(len(tests)):
    print(" ")
    print(tests[t])
    print(power[t])

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

corrupt proportion : 0.75
sample size : (200, 400, 600, 800, 1000)
 
<function hsicfuse_test at 0x7f38a0028b80>
[0.96 1.   1.   1.   1.  ]
 
<function hsic_test at 0x7f370d8c3790>
[0.14 0.14 0.12 0.11 0.09]
 
<function hsicagginc1_test at 0x7f370d8c3940>
[0.06       0.05       0.09       0.07       0.14999999]
 
<function hsicagginc100_test at 0x7f370d86edc0>
[0.24       0.59999996 0.82       0.93       0.94      ]
 
<function hsicagginc200_test at 0x7f370d86ee50>
[0.24       0.74       0.90999997 0.96       0.98999995]
 
<function hsicaggincquad_test at 0x7f370d86eee0>
[0.24 0.74 0.94 1.   1.  ]


### MNIST vary C

In [13]:
N        = 500
rep      = 100
C_values = (0.6, 0.7, 0.8, 0.9)

tests = (hsicfuse_test, hsic_test, hsicagginc1_test, hsicagginc100_test, hsicagginc200_test, hsicaggincquad_test)
outputs_mnist_n = jnp.zeros((len(tests), len(N_values), rep))
outputs_mnist_n = outputs_mnist_n.tolist()
key = random.PRNGKey(42)
seed = 42
for r in tqdm(range(len(C_values))):
    corrupt_proportion = C_values[r]
    for i in tqdm(range(rep)):
        key, subkey = random.split(key)
        X, Y = ind_X_Y_corrupt(key, N, corrupt_proportion)
        key, subkey = random.split(key)
        seed += 1
        for j in range(len(tests)):
            test = tests[j]
            outputs_mnist_n[j][r][i] = test(X, Y, subkey, seed)

outputs_mnist_n = jnp.array(outputs_mnist_n)   
power = jnp.array([[jnp.mean(outputs_mnist_n[j][r])for r in range(len(C_values))] for j in range(len(tests))])
np.save("results/mnist_c.npy", power)
np.save("results/mnist_c_x_axis.npy", N_values)

print("corrupt proportion :", C_values)
print("sample size :", N)
for t in range(len(tests)):
    print(" ")
    print(tests[t])
    print(power[t])

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

corrupt proportion : 0.9
sample size : (200, 400, 600, 800, 1000)
 
<function hsicfuse_test at 0x7f38a0028b80>
[1.   1.   1.   0.61]
 
<function hsic_test at 0x7f370d8c3790>
[0.21       0.09999999 0.07       0.06      ]
 
<function hsicagginc1_test at 0x7f370d8c3940>
[0.09999999 0.04       0.08       0.01      ]
 
<function hsicagginc100_test at 0x7f370d86edc0>
[0.96999997 0.89       0.37       0.02      ]
 
<function hsicagginc200_test at 0x7f370d86ee50>
[1.   0.95 0.48 0.06]
 
<function hsicaggincquad_test at 0x7f370d86eee0>
[1.   0.96 0.48 0.06]
